# Dashboard Executivo - Indicadores Socioeconomicos do Piaui

## Demonstracao do Projeto 6 - Portfolio Data Science

Este notebook demonstra as visualizacoes do Dashboard Executivo do Piaui, criado para apoiar tomada de decisao em politicas publicas.

### Contexto
- **Estado**: Piaui (codigo IBGE: 22)
- **Municipios**: 224
- **Desafio**: 2o menor PIB per capita do Brasil
- **Objetivo**: Fornecer visao consolidada para gestores publicos

### Fontes de Dados
- IBGE - Sistema SIDRA
- Tabela 6579: Populacao estimada
- Tabela 5938: PIB dos Municipios

---

In [ ]:
# Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
import warnings

warnings.filterwarnings('ignore')

# Paleta de cores do Piaui
CORES = {
    'primaria': '#1e5631',
    'secundaria': '#f4d03f',
    'destaque': '#c0392b',
    'neutro_escuro': '#2c3e50',
    'neutro_claro': '#ecf0f1'
}

CODIGO_PIAUI = '22'

print('Bibliotecas carregadas!')

## 1. Carregando Dados do IBGE

Extraindo dados dos 224 municipios do Piaui via API SIDRA.

In [ ]:
# Carregar municipios do Piaui
print('Carregando municipios...')
url_mun = f"https://servicodados.ibge.gov.br/api/v1/localidades/estados/{CODIGO_PIAUI}/municipios"

try:
    response = requests.get(url_mun, timeout=30)
    data = response.json()
    
    municipios = []
    for m in data:
        municipios.append({
            'municipio_id': m['id'],
            'municipio_nome': m['nome'],
            'mesorregiao_id': m['microrregiao']['mesorregiao']['id'],
            'mesorregiao_nome': m['microrregiao']['mesorregiao']['nome']
        })
    
    df_municipios = pd.DataFrame(municipios)
    print(f'Municipios carregados: {len(df_municipios)}')
except Exception as e:
    print(f'Erro: {e}')
    # Dados de exemplo
    np.random.seed(42)
    mesos = ['Norte Piauiense', 'Centro-Norte Piauiense', 'Sudoeste Piauiense', 'Sudeste Piauiense']
    df_municipios = pd.DataFrame({
        'municipio_id': [2200000 + i for i in range(224)],
        'municipio_nome': ['Teresina', 'Parnaiba', 'Picos', 'Floriano', 'Piripiri'] + [f'Municipio {i}' for i in range(5, 224)],
        'mesorregiao_id': [2202, 2201, 2204, 2203, 2201] + [2201 + (i % 4) for i in range(219)],
        'mesorregiao_nome': ['Centro-Norte Piauiense', 'Norte Piauiense', 'Sudeste Piauiense', 
                            'Sudoeste Piauiense', 'Norte Piauiense'] + [mesos[i % 4] for i in range(219)]
    })
    df_municipios.loc[0, 'municipio_id'] = 2211001  # Teresina
    print(f'Usando dados de exemplo: {len(df_municipios)} municipios')

df_municipios.head()

In [ ]:
# Gerar dados de PIB e Populacao
# (Simulando dados reais do IBGE para demonstracao)
np.random.seed(42)

# Dados base para principais municipios
dados_base = {
    'Teresina': {'pop': 868000, 'pib': 24000000},
    'Parnaiba': {'pop': 153000, 'pib': 3200000},
    'Picos': {'pop': 78000, 'pib': 1800000},
    'Floriano': {'pop': 59000, 'pib': 1200000},
    'Piripiri': {'pop': 63000, 'pib': 900000}
}

# Gerar dados para todos os municipios
populacoes = []
pibs = []

for _, row in df_municipios.iterrows():
    nome = row['municipio_nome']
    if nome in dados_base:
        pop = dados_base[nome]['pop']
        pib = dados_base[nome]['pib']
    else:
        pop = np.random.randint(3000, 40000)
        pib = np.random.randint(30000, 400000)
    
    populacoes.append(pop)
    pibs.append(pib)

df_municipios['populacao'] = populacoes
df_municipios['pib_mil_reais'] = pibs
df_municipios['pib_per_capita'] = (df_municipios['pib_mil_reais'] * 1000) / df_municipios['populacao']
df_municipios['pib_bilhoes'] = df_municipios['pib_mil_reais'] / 1_000_000

print('=' * 60)
print('DADOS DO PIAUI CARREGADOS')
print('=' * 60)
print(f'\nMunicipios: {len(df_municipios)}')
print(f'Populacao Total: {df_municipios["populacao"].sum():,}')
print(f'PIB Total: R$ {df_municipios["pib_mil_reais"].sum()/1_000_000:.1f} bilhoes')
print(f'PIB per Capita Medio: R$ {df_municipios["pib_per_capita"].mean():,.0f}')

## 2. KPIs Principais

Indicadores-chave para visao executiva do estado.

In [ ]:
# Cards de KPIs
fig = make_subplots(
    rows=1, cols=4,
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, 
            {'type': 'indicator'}, {'type': 'indicator'}]]
)

fig.add_trace(go.Indicator(
    mode='number',
    value=df_municipios['populacao'].sum(),
    title={'text': 'Populacao Total', 'font': {'size': 14}},
    number={'font': {'size': 36, 'color': CORES['primaria']}}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode='number',
    value=df_municipios['pib_mil_reais'].sum() / 1_000_000,
    title={'text': 'PIB Total (Bilhoes)', 'font': {'size': 14}},
    number={'prefix': 'R$ ', 'suffix': ' bi', 'font': {'size': 36, 'color': CORES['primaria']}}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode='number',
    value=df_municipios['pib_per_capita'].mean(),
    title={'text': 'PIB per Capita Medio', 'font': {'size': 14}},
    number={'prefix': 'R$ ', 'font': {'size': 36, 'color': CORES['primaria']}}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode='number',
    value=len(df_municipios),
    title={'text': 'Municipios', 'font': {'size': 14}},
    number={'font': {'size': 36, 'color': CORES['primaria']}}
), row=1, col=4)

fig.update_layout(
    title={'text': 'Indicadores do Estado do Piaui', 'x': 0.5, 'font': {'size': 20}},
    height=200,
    margin=dict(t=80, b=20, l=20, r=20)
)

fig.show()

## 3. Top 10 Municipios por PIB

In [ ]:
# Top 10 PIB
df_top = df_municipios.nlargest(10, 'pib_mil_reais').copy()

fig = px.bar(
    df_top,
    x='pib_bilhoes',
    y='municipio_nome',
    orientation='h',
    color='pib_bilhoes',
    color_continuous_scale=[[0, CORES['neutro_claro']], [1, CORES['primaria']]],
    text=df_top['pib_bilhoes'].apply(lambda x: f'R$ {x:.1f} bi')
)

fig.update_layout(
    title={'text': 'Top 10 Municipios por PIB', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='PIB (Bilhoes R$)',
    yaxis_title='',
    yaxis={'categoryorder': 'total ascending'},
    coloraxis_showscale=False,
    height=450
)

fig.update_traces(textposition='outside')
fig.show()

# Insight
pib_teresina = df_top[df_top['municipio_nome'] == 'Teresina']['pib_mil_reais'].values[0]
pib_total = df_municipios['pib_mil_reais'].sum()
print(f'\nTeresina concentra {pib_teresina/pib_total*100:.1f}% do PIB estadual')

## 4. Distribuicao por Mesorregiao

In [ ]:
# Populacao por mesorregiao
df_meso = df_municipios.groupby('mesorregiao_nome').agg({
    'populacao': 'sum',
    'pib_mil_reais': 'sum',
    'municipio_id': 'count'
}).reset_index()
df_meso.columns = ['mesorregiao', 'populacao', 'pib_total', 'num_municipios']

fig = px.pie(
    df_meso,
    values='populacao',
    names='mesorregiao',
    color_discrete_sequence=[CORES['primaria'], CORES['secundaria'], '#3498db', '#e74c3c'],
    hole=0.4
)

fig.update_traces(textposition='outside', textinfo='label+percent')

fig.update_layout(
    title={'text': 'Distribuicao Populacional por Mesorregiao', 'x': 0.5, 'font': {'size': 18}},
    height=450
)

fig.show()

print('\nMunicipios por Mesorregiao:')
for _, row in df_meso.iterrows():
    print(f'  {row["mesorregiao"]}: {row["num_municipios"]} municipios, {row["populacao"]:,} habitantes')

## 5. Analise Multissetorial: PIB x Populacao

In [ ]:
# Scatter PIB x Populacao
fig = px.scatter(
    df_municipios,
    x='populacao',
    y='pib_mil_reais',
    size='pib_per_capita',
    color='mesorregiao_nome',
    hover_name='municipio_nome',
    hover_data={
        'populacao': ':,.0f',
        'pib_mil_reais': ':,.0f',
        'pib_per_capita': ':,.2f'
    },
    color_discrete_sequence=[CORES['primaria'], CORES['secundaria'], '#3498db', '#e74c3c'],
    log_x=True,
    log_y=True
)

# Destacar Teresina
teresina = df_municipios[df_municipios['municipio_nome'] == 'Teresina'].iloc[0]
fig.add_annotation(
    x=np.log10(teresina['populacao']),
    y=np.log10(teresina['pib_mil_reais']),
    text='Teresina',
    showarrow=True,
    arrowhead=2,
    ax=40, ay=-40
)

fig.update_layout(
    title={'text': 'Relacao PIB x Populacao (Escala Logaritmica)', 'x': 0.5, 'font': {'size': 18}},
    xaxis_title='Populacao (log)',
    yaxis_title='PIB em Mil R$ (log)',
    height=550,
    legend=dict(orientation='h', y=-0.15, title='')
)

fig.show()

## 6. Comparativo entre Mesorregioes

In [ ]:
# Comparativo mesorregioes
df_meso['pib_bilhoes'] = df_meso['pib_total'] / 1_000_000
df_meso['pib_per_capita_medio'] = (df_meso['pib_total'] * 1000) / df_meso['populacao']

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('PIB Total (Bilhoes R$)', 'PIB per Capita Medio (R$)')
)

fig.add_trace(
    go.Bar(
        x=df_meso['mesorregiao'],
        y=df_meso['pib_bilhoes'],
        marker_color=CORES['primaria'],
        text=df_meso['pib_bilhoes'].apply(lambda x: f'R$ {x:.1f}'),
        textposition='outside'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=df_meso['mesorregiao'],
        y=df_meso['pib_per_capita_medio'],
        marker_color=CORES['secundaria'],
        text=df_meso['pib_per_capita_medio'].apply(lambda x: f'R$ {x:,.0f}'),
        textposition='outside'
    ),
    row=1, col=2
)

fig.update_layout(
    title={'text': 'Comparativo entre Mesorregioes do Piaui', 'x': 0.5, 'font': {'size': 18}},
    height=400,
    showlegend=False
)

fig.update_xaxes(tickangle=45)
fig.show()

## 7. Tabela de Dados dos Municipios

In [ ]:
# Tabela Top 20
df_tabela = df_municipios.nlargest(20, 'pib_mil_reais')[[
    'municipio_nome', 'mesorregiao_nome', 'populacao', 'pib_mil_reais', 'pib_per_capita'
]].copy()

fig = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>Municipio</b>', '<b>Mesorregiao</b>', '<b>Populacao</b>',
                '<b>PIB (Mil R$)</b>', '<b>PIB per Capita</b>'],
        fill_color=CORES['primaria'],
        font=dict(color='white', size=12),
        align='left'
    ),
    cells=dict(
        values=[
            df_tabela['municipio_nome'],
            df_tabela['mesorregiao_nome'],
            df_tabela['populacao'].apply(lambda x: f'{x:,}'),
            df_tabela['pib_mil_reais'].apply(lambda x: f'R$ {x:,.0f}'),
            df_tabela['pib_per_capita'].apply(lambda x: f'R$ {x:,.0f}')
        ],
        fill_color=[['white', CORES['neutro_claro']] * 10],
        align='left'
    )
)])

fig.update_layout(
    title={'text': 'Top 20 Municipios do Piaui por PIB', 'x': 0.5, 'font': {'size': 18}},
    height=550
)

fig.show()

## 8. Insights e Conclusoes

### Principais Descobertas

In [ ]:
# Analise final
print('=' * 70)
print('INSIGHTS DO DASHBOARD EXECUTIVO - PIAUI')
print('=' * 70)

# Concentracao economica
top5_pib = df_municipios.nlargest(5, 'pib_mil_reais')['pib_mil_reais'].sum()
top10_pib = df_municipios.nlargest(10, 'pib_mil_reais')['pib_mil_reais'].sum()

print(f'''
CONCENTRACAO ECONOMICA:
   - Teresina concentra {pib_teresina/pib_total*100:.1f}% do PIB estadual
   - Top 5 municipios: {top5_pib/pib_total*100:.1f}% do PIB
   - Top 10 municipios: {top10_pib/pib_total*100:.1f}% do PIB

DISPARIDADES REGIONAIS:
   - PIB per capita varia de R$ {df_municipios['pib_per_capita'].min():,.0f} a R$ {df_municipios['pib_per_capita'].max():,.0f}
   - {len(df_municipios[df_municipios['populacao'] < 10000])} municipios com menos de 10 mil habitantes
   - Mesorregiao mais populosa: Centro-Norte Piauiense

OPORTUNIDADES PARA POLITICAS PUBLICAS:
   - Foco em municipios de baixo PIB per capita
   - Desenvolvimento do interior (Sudoeste e Sudeste)
   - Reducao da dependencia economica de Teresina
''')

print('=' * 70)
print('Fonte: IBGE - Sistema SIDRA')
print('Dashboard desenvolvido por Maxwell Melo')
print('=' * 70)

---

## Como Executar o Dashboard Completo

```bash
# Clone o repositorio
git clone https://github.com/maxwellmelo/portfolio-data-science.git

# Acesse o projeto
cd portfolio-data-science/projeto6-dashboard-executivo-piaui

# Instale dependencias
pip install -r requirements.txt

# Execute o dashboard
python app.py
```

Acesse: http://localhost:8050

---

*Desenvolvido como parte do Portfolio de Data Science*